In [2]:
import pandas as pd
import numpy as np
import os
import json
import gc
import re
# import networkx as nx
from datetime import datetime
import time
import preprocessor as p
import ast
from wordsegment import load, segment
import random
load()
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import unicodedata
from sklearn.model_selection import train_test_split
from nltk.stem import PorterStemmer, WordNetLemmatizer
from collections import Counter
from scipy.stats import entropy, shapiro, mannwhitneyu
from numpy.linalg import norm
import numpy as np
from spacy.lang.hi import STOP_WORDS as STOP_WORDS_HI
from spacy.lang.de import STOP_WORDS as STOP_WORDS_DE
from spacy.lang.es import STOP_WORDS as STOP_WORDS_ES
import string

/home/sarahm/anaconda3/envs/trans_env/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
stopwords2 = ['तुम','मेरी','मुझे','क्योंकि','हम','प्रति','अबकी','दे',
                              'आगे','', 'अर्थात', 'कुछ', 'तेरी', 'साबुत', 'अपनि', 'हूं',
                              'काफि', 'यिह', 'जा' ,'दे', 'देकर' ,'रह', 'कह' , 'कर' ,
                              'कहा', 'बात' , 'जिन्हों', 'किर', 'कोई','माननीय','शहर','बताएं',
                              'कौनसी','क्लिक','किसकी','बड़े','मैं','and','रही','आज','लें',
                              'आपके','मिलकर','सब','मेरे','जी','श्री','वैसा','आपका','अंदर',
                              'अत', 'अपना', 'अपनी', 'अपने', 'अभी', 'आदि', 'आप', 'इत्यादि',
                              'इन', 'इनका', 'इन्हीं', 'इन्हें', 'इन्हों', 'इस', 'इसका', 'इसकी', 
                              'इसके', 'इसमें', 'इसी', 'इसे', 'उन', 'उनका', 'उनकी', 'उनके',
                              'उनको', 'उन्हीं', 'उन्हें', 'उन्हों', 'उस', 'उसके', 'उसी', 'उसे',
                              'एक', 'एवं', 'एस', 'ऐसे', 'और', 'कई', 'कर','करता', 'करते',
                              'करना', 'करने', 'करें', 'कहते', 'कहा', 'का', 'काफ़ी', 'कि',
                              'कितना', 'किन्हें', 'किन्हों', 'किया', 'किर', 'किस', 'किसी',
                              'किसे', 'की', 'कुछ', 'कुल', 'के', 'को', 'कोई', 'कौन',
                              'कौनसा', 'गया', 'घर', 'जब', 'जहाँ', 'जा', 'जितना', 'जिन',
                              'जिन्हें', 'जिन्हों', 'जिस', 'जिसे', 'जीधर', 'जैसा', 'जैसे', 'जो',
                              'तक', 'तब', 'तरह', 'तिन', 'तिन्हें', 'तिन्हों', 'तिस',
                              'तिसे', 'तो', 'था', 'थी', 'थे', 'दबारा', 'दिया', 'दुसरा', 
                              'दूसरे', 'दो', 'द्वारा', 'न', 'नहीं', 'ना', 'निहायत', 'नीचे',
                              'ने', 'पर', 'पर', 'पहले', 'पूरा', 'पे', 'फिर', 'बनी', 'बही',
                              'बहुत', 'बाद', 'बाला', 'बिलकुल', 'भी', 'भीतर', 'मगर', 'मानो',
                              'मे', 'में', 'यदि', 'यह', 'यहाँ', 'यही', 'या', 'यिह', 'ये', 'रखें',
                              'रहा', 'रहे', 'ऱ्वासा', 'लिए', 'लिये', 'लेकिन', 'व', 'वर्ग', 'वह',
                              'वह', 'वहाँ', 'वहीं', 'वाले', 'वुह', 'वे', 'वग़ैरह', 'संग', 'सकता',
                              'सकते', 'सबसे', 'सभी', 'साथ', 'साबुत', 'साभ', 'सारा', 'से', 'सो',
                              'ही', 'हुआ', 'हुई', 'हुए', 'है', 'हैं', 'हो', 'होता', 'होती', 'होते',
                              'होना', 'होने', 'अपनि', 'जेसे', 'होति', 'सभि', 'तिंहों', 'इंहों', 'दवारा',
                              'इसि', 'किंहें', 'थि', 'उंहों', 'ओर', 'जिंहें', 'वहिं', 'अभि', 'बनि', 'हि',
                              'उंहिं', 'उंहें', 'हें', 'वगेरह', 'एसे', 'रवासा', 'कोन', 'निचे', 'काफि',
                              'उसि', 'पुरा', 'भितर', 'हे', 'बहि', 'वहां', 'कोइ', 'यहां', 'जिंहों', 
                              'तिंहें', 'किसि', 'कइ', 'यहि', 'इंहिं', 'जिधर', 'इंहें', 'अदि', 'इतयादि',
                              'हुइ', 'कोनसा', 'इसकि', 'दुसरे', 'जहां', 'अप', 'किंहों', 'उनकि', 'भि', 
                              'वरग', 'हुअ', 'जेसा', 'नहिं']

In [4]:
STOP_LIST = set(stopwords.words('english'))
with open('/home/sarahm/LCS2-Hate-Speech-Detection-Diffusion/mallet_stoplist.txt',"r") as f:         
        MALLET_STOPLIST = f.readlines()
MALLET_STOPLIST = [x.replace("\n", "") for x in MALLET_STOPLIST] 
STOP_LIST = STOP_LIST.union(set(MALLET_STOPLIST))
STOP_LIST = list(STOP_LIST)
STOP_LIST.extend(STOP_WORDS_HI)
STOP_LIST.extend(STOP_WORDS_DE)
STOP_LIST.extend(STOP_WORDS_ES)
STOP_LIST.extend(["rt",":","…","&amp","","https://t.co/…"])
STOP_LIST += stopwords2
stemmer = PorterStemmer()
table = str.maketrans(dict.fromkeys(string.punctuation))  # OR {key: None for key in string.punctuation}

def preprocess(text_string):
    text_string = text_string.strip().lower()
    space_pattern = '\s+'
    mention_regex = '@[\w\-]+'
    parsed_text = re.sub(space_pattern, ' ', text_string)
    parsed_text = re.sub(mention_regex, ' ', parsed_text)
    text_string = re.sub(r'^https?:\/\/.*[\r\n]*', '', parsed_text, flags=re.MULTILINE) 
    parsed_text = parsed_text.translate(table)
    parsed_text = parsed_text.strip()
    return parsed_text

def tokenize(tweet):
    """Removes punctuation & excess whitespace, sets to lowercase,
    and stems tweets. Returns a list of stemmed tokens."""
    tokens = []
    for word in tweet.split():
        if word in STOP_LIST:
            continue
        tokens.append(word)
    return tokens


def preprocess_dataset(dataset,prune=False,high_prune=500,low_prune=750): 
    words = [] 
    for data in dataset : 
        tokens = tokenize(preprocess(data))
        words.extend(tokens)
    if prune:
        mc = Counter(words).most_common()
        vocab_prune=[]
        for word,count in mc[:high_prune]:
            vocab_prune.extend([word]*count)
        for word,count in mc[-low_prune:]:
            vocab_prune.extend([word]*count)
        return vocab_prune
    else:
        return words
    
def get_combined_df(base):
    INPUT_FOLDER = "/home/sarahm/LCS2-Hate-Speech-Detection-Diffusion/dataset/ORG_DATASETS/"
    train_df = pd.read_csv(INPUT_FOLDER+base+"/train_final.csv",lineterminator="\n")
    test_df = pd.read_csv(INPUT_FOLDER+base+"/test_final.csv",lineterminator="\n")
    val_df = pd.read_csv(INPUT_FOLDER+base+"/val_final.csv",lineterminator="\n")
    df_combined = pd.concat([train_df, val_df, test_df]).reset_index(drop=True)
    df_combined = df_combined.sample(frac=1,random_state=42).reset_index(drop=True)
    print(df_combined.shape,train_df.shape,val_df.shape,test_df.shape)
    return df_combined, train_df, val_df, test_df

def JSD(P, Q):
    _P = P / norm(P, ord=1)
    _Q = Q / norm(Q, ord=1)
    _M = 0.5 * (_P + _Q)
    return 0.5 * (entropy(_P, _M) + entropy(_Q, _M))
    
def for_4_classes(BASE,prune=True):
    df, train_df, val_df, test_df = get_combined_df(BASE)
    
    column = 'clean_text'
#     if BASE=="HOPN":
#         column="text"
        
    hate_dataset = preprocess_dataset(df[df['label'] == 0][column].values.tolist(),prune=prune)
    print("hate_dataset size: ", len(hate_dataset))
    offensive_dataset = preprocess_dataset(df[df['label'] == 1][column].values.tolist(),prune=prune)
    print("offensive_dataset size: ", len(offensive_dataset))
    provocative_dataset = preprocess_dataset(df[df['label'] == 2][column].values.tolist(),prune=prune)
    print("provocative_dataset size: ", len(provocative_dataset))
    control_dataset = preprocess_dataset(df[df['label'] == 3][column].values.tolist(),prune=prune)
    print("control_dataset size: ", len(control_dataset))

    hate_set = set(hate_dataset)
    print("hate_set size: ", len(hate_set))
    offensive_set = set(offensive_dataset)
    print("offensive_set size: ", len(offensive_set))
    provocative_set = set(provocative_dataset)
    print("provocative_set size: ", len(provocative_set))
    control_set = set(control_dataset)
    print("control_set size: ", len(control_set))
    union_set = control_set.union(provocative_set.union(offensive_set.union(hate_set)))
    print("union_set size: ", len(union_set))

    hate_counter = Counter(hate_dataset)
    offensive_counter = Counter(offensive_dataset)
    provocative_counter = Counter(provocative_dataset)
    control_counter = Counter(control_dataset)

    pdf_hate = []
    pdf_offensive = []
    pdf_provocative = []
    pdf_control = []
    pdf_hate_raw = []
    pdf_offensive_raw = []
    pdf_provocative_raw = []
    pdf_control_raw = []

    N = len(hate_dataset) + len(offensive_dataset) + len(provocative_dataset) + len(control_dataset)
    V = len(union_set)

    for word in union_set : 
        if word in hate_set : 
            pdf_hate.append((hate_counter[word]+1)/(len(hate_dataset) + len(hate_set)))
        else : 
            pdf_hate.append(1/(N+V))
        pdf_hate_raw.append(hate_counter.get(word,0))
        if word in offensive_set : 
            pdf_offensive.append((offensive_counter[word]+1)/(len(offensive_dataset) + len(offensive_set)))
        else : 
            pdf_offensive.append(1/(N+V))
        pdf_offensive_raw.append(offensive_counter.get(word,0))
        if word  in provocative_set : 
            pdf_provocative.append((provocative_counter[word]+1)/(len(provocative_dataset) + len(provocative_set)))
        else : 
            pdf_provocative.append(1/(N+V))
        pdf_provocative_raw.append(provocative_counter.get(word,0))    
        if word in control_set : 
            pdf_control.append((control_counter[word]+1)/(len(control_dataset) + len(control_set)))
        else : 
            pdf_control.append(1/(N+V))
        pdf_control_raw.append(control_counter.get(word,0))  
    print(sum(pdf_hate))
    print(sum(pdf_offensive))
    print(sum(pdf_provocative))
    print(sum(pdf_control))
    
    print("\n\n ---- JS")
    print("Jenson Shannon Distance for H O", JSD(pdf_hate, pdf_offensive))
    print("Jenson Shannon Distance for H P", JSD(pdf_hate, pdf_provocative))
    print("Jenson Shannon Distance for H N", JSD(pdf_hate, pdf_control))
    print("Jenson Shannon Distance for O P", JSD(pdf_offensive, pdf_provocative))
    print("Jenson Shannon Distance for O N", JSD(pdf_offensive, pdf_control))
    print("Jenson Shannon Distance for P N", JSD(pdf_provocative, pdf_control))
    
#     print("\n\n ---- MW")
#     print("2 way MW TEST for H O:: ", mannwhitneyu(pdf_hate_raw, pdf_offensive_raw))
#     print("2 way MW TEST  for H P:: ", mannwhitneyu(pdf_hate_raw, pdf_provocative_raw))
#     print("2 way MW TEST  for H N:: ", mannwhitneyu(pdf_hate_raw, pdf_control_raw))
#     print("2 way MW TEST  for O P:: ", mannwhitneyu(pdf_offensive_raw, pdf_provocative_raw))
#     print("2 way MW TEST  for O N:: ", mannwhitneyu(pdf_offensive_raw, pdf_control_raw))
#     print("2 way MW TEST for P N:: ", mannwhitneyu(pdf_provocative_raw, pdf_control_raw))
    
def for_3_classes(BASE,prune=True):
    df, train_df, val_df, test_df = get_combined_df(BASE)
    column = 'clean_text'

    hate_dataset = preprocess_dataset(df[df['label'] == 0][column].values.tolist(),prune=prune)
    print("hate_dataset size: ", len(hate_dataset))
    offensive_dataset = preprocess_dataset(df[df['label'] == 1][column].values.tolist(),prune=prune)
    print("offensive_dataset size: ", len(offensive_dataset))
    control_dataset = preprocess_dataset(df[df['label'] == 2][column].values.tolist(),prune=prune)
    print("control_dataset size: ", len(control_dataset))

    hate_set = set(hate_dataset)
    print("hate_set size: ", len(hate_set))
    offensive_set = set(offensive_dataset)
    control_set = set(control_dataset)
    print("control_set size: ", len(control_set))
    union_set = control_set.union((offensive_set.union(hate_set)))
    print("union_set size: ", len(union_set))

    hate_counter = Counter(hate_dataset)
    offensive_counter = Counter(offensive_dataset)
    control_counter = Counter(control_dataset)

    pdf_hate = []
    pdf_offensive = []
    pdf_control = []

    pdf_hate_raw = []
    pdf_offensive_raw=[]
    pdf_control_raw = []
    N = len(hate_dataset) + len(offensive_dataset) + len(control_dataset)
    V = len(union_set)

    for word in union_set : 
        if word in hate_set : 
            pdf_hate.append((hate_counter[word]+1)/(len(hate_dataset) + len(hate_set)))
        else : 
            pdf_hate.append(1/(N+V))
        pdf_hate_raw.append(hate_counter.get(word,0))
        if word in offensive_set : 
            pdf_offensive.append((offensive_counter[word]+1)/(len(offensive_dataset) + len(offensive_set)))
        else : 
            pdf_offensive.append(1/(N+V))
        pdf_offensive_raw.append(offensive_counter.get(word,0))
        if word in control_set : 
            pdf_control.append((control_counter[word]+1)/(len(control_dataset) + len(control_set)))
        else : 
            pdf_control.append(1/(N+V))
        pdf_control_raw.append(control_counter.get(word,0)) 

    print(sum(pdf_hate))
    print(sum(pdf_offensive))
    print(sum(pdf_control))
    
    print("\n\n ---- JS")
    print("Jenson Shannon Distance for H O", JSD(pdf_hate, pdf_offensive))
    print("Jenson Shannon Distance for H N", JSD(pdf_hate, pdf_control))
    print("Jenson Shannon Distance for O N", JSD(pdf_offensive, pdf_control))
#     print("\n\n ---- MW")
#     print("2 way MW TEST  for H O:: ", mannwhitneyu(pdf_hate_raw, pdf_offensive_raw))
#     print("2 way MW TEST  for H N:: ", mannwhitneyu(pdf_hate_raw, pdf_control_raw))
#     print("2 way MW TEST  for O N:: ", mannwhitneyu(pdf_offensive_raw, pdf_control_raw))
    
def for_other_datasets(BASE,prune=True):
    df, train_df, val_df, test_df = get_combined_df(BASE)
    column = 'clean_text'

    hate_dataset = preprocess_dataset(df[df['label'] == 0][column].values.tolist(),prune=prune)
    print("hate_dataset size: ", len(hate_dataset))
    control_dataset = preprocess_dataset(df[df['label'] == 1][column].values.tolist(),prune=prune)
    print("control_dataset size: ", len(control_dataset))

    hate_set = set(hate_dataset)
    print("hate_set size: ", len(hate_set))
    control_set = set(control_dataset)
    print("control_set size: ", len(control_set))
    union_set = control_set.union(hate_set)
    print("union_set size: ", len(union_set))

    hate_counter = Counter(hate_dataset)
    control_counter = Counter(control_dataset)

    pdf_hate = []
    pdf_control = []
    pdf_hate_raw = []
    pdf_control_raw = []
    N = len(hate_dataset) + len(control_dataset)
    V = len(union_set)

    for word in union_set : 
        if word in hate_set : 
            pdf_hate.append((hate_counter[word]+1)/(len(hate_dataset) + len(hate_set)))
        else : 
            pdf_hate.append(1/(N+V))
        pdf_hate_raw.append(hate_counter.get(word,0))
        if word in control_set : 
            pdf_control.append((control_counter[word]+1)/(len(control_dataset) + len(control_set)))
        else : 
            pdf_control.append(1/(N+V))
        pdf_control_raw.append(control_counter.get(word,0)) 
    print(sum(pdf_hate))
    print(sum(pdf_control))

    print("\n\n ---- JS")
    print("Jenson Shannon Distance for H N", JSD(pdf_hate, pdf_control))
#     print("\n\n ---- MW")
#     print("2 way MW TEST  for H N:: ", mannwhitneyu(pdf_hate_raw, pdf_control_raw))


def binary_compare(BASE,multi=True,prune=True):
    df, train_df, val_df, test_df = get_combined_df(BASE)
    column = 'clean_text'
#     if BASE=="HOPN":
#         column="text"
    num_unq = len(df["label"].unique())
    if num_unq >2 and multi:
        print("Here")
        hate_dataset = preprocess_dataset(df[df['label'] <=1][column].values.tolist(),prune=prune)
        print("hate_dataset size: ", len(hate_dataset))
        control_dataset = preprocess_dataset(df[df['label'] > 1][column].values.tolist(),prune=prune)
        print("control_dataset size: ", len(control_dataset))
    else:
        hate_dataset = preprocess_dataset(df[df['label'] == 0][column].values.tolist(),prune=prune)
        print("hate_dataset size: ", len(hate_dataset))
        control_dataset = preprocess_dataset(df[df['label'] == 1][column].values.tolist(),prune=prune)
        print("control_dataset size: ", len(control_dataset))
    hate_set = set(hate_dataset)
    print("hate_set size: ", len(hate_set))
    control_set = set(control_dataset)
    print("control_set size: ", len(control_set))
    union_set = control_set.union(hate_set)
    print("union_set size: ", len(union_set))

    hate_counter = Counter(hate_dataset)
    control_counter = Counter(control_dataset)

    pdf_hate = []
    pdf_control = []
    pdf_hate_raw = []
    pdf_control_raw = []
    N = len(hate_dataset) + len(control_dataset)
    V = len(union_set)

    for word in union_set : 
        if word in hate_set : 
            pdf_hate.append((hate_counter[word]+1)/(len(hate_dataset) + len(hate_set)))
        else : 
            pdf_hate.append(1/(N+V))
        pdf_hate_raw.append(hate_counter.get(word,0))
        if word in control_set : 
            pdf_control.append((control_counter[word]+1)/(len(control_dataset) + len(control_set)))
        else : 
            pdf_control.append(1/(N+V))
        pdf_control_raw.append(control_counter.get(word,0)) 
    print(sum(pdf_hate))
    print(sum(pdf_control))

    print("\n\n ---- JS")
    print("Jenson Shannon Distance for H N", JSD(pdf_hate, pdf_control))

In [5]:
for_4_classes(BASE="HOPN",prune=True)

(51367, 20) (41093, 20) (5137, 20) (5137, 20)
hate_dataset size:  45999
offensive_dataset size:  88423
provocative_dataset size:  110227
control_dataset size:  260302
hate_set size:  1250
offensive_set size:  1250
provocative_set size:  1250
control_set size:  1250
union_set size:  3799
1.005010319410342
1.0050103194103392
1.0050103194103484
1.0050103194103286


 ---- JS
Jenson Shannon Distance for H O 0.1354406709671649
Jenson Shannon Distance for H P 0.0872160407967375
Jenson Shannon Distance for H N 0.11819917708809587
Jenson Shannon Distance for O P 0.11858631855534867
Jenson Shannon Distance for O N 0.12101396453080641
Jenson Shannon Distance for P N 0.06379174493749891


In [6]:
for_4_classes(BASE="Founta")

(59189, 4) (37880, 4) (9471, 4) (11838, 4)
hate_dataset size:  9982
offensive_dataset size:  40077
provocative_dataset size:  26106
control_dataset size:  105562
hate_set size:  1250
offensive_set size:  1250
provocative_set size:  1250
control_set size:  1250
union_set size:  4059
1.0151195461445035
1.0151195461445022
1.0151195461445306
1.0151195461444988


 ---- JS
Jenson Shannon Distance for H O 0.20482237651287338
Jenson Shannon Distance for H P 0.46707207975029263
Jenson Shannon Distance for H N 0.3187275856004806
Jenson Shannon Distance for O P 0.46786129698656664
Jenson Shannon Distance for O N 0.31811113178723754
Jenson Shannon Distance for P N 0.2431742246454875


In [7]:
for_3_classes(BASE="davidson")

(24783, 4) (15860, 4) (3966, 4) (4957, 4)
hate_dataset size:  6449
offensive_dataset size:  72641
control_dataset size:  13522
hate_set size:  1250
control_set size:  1250
union_set size:  3058
1.0188982962265793
1.0188982962266055
1.0188982962266018


 ---- JS
Jenson Shannon Distance for H O 0.20446479111199295
Jenson Shannon Distance for H N 0.34693070497570677
Jenson Shannon Distance for O N 0.3320126371438755


In [8]:
for_other_datasets(BASE="HASOC19")

(17657, 4) (11468, 4) (2868, 4) (3321, 4)
hate_dataset size:  31733
control_dataset size:  36899
hate_set size:  1250
control_set size:  1250
union_set size:  2227
1.0137879450740026
1.0137879450740164


 ---- JS
Jenson Shannon Distance for H N 0.24638369980651817


In [9]:
for_other_datasets(BASE="OLID")

(14100, 4) (10592, 4) (2648, 4) (860, 4)
hate_dataset size:  21781
control_dataset size:  35673
hate_set size:  1250
control_set size:  1250
union_set size:  2123
1.0146533058059275
1.014653305805919


 ---- JS
Jenson Shannon Distance for H N 0.11546945139757275


In [10]:
for_other_datasets(BASE="HateEval")

(19600, 4) (13500, 4) (1500, 4) (4600, 4)
hate_dataset size:  36475
control_dataset size:  40672
hate_set size:  1250
control_set size:  1250
union_set size:  2140
1.011225043197495
1.0112250431974938


 ---- JS
Jenson Shannon Distance for H N 0.13068339166855147


In [60]:
for_other_datasets(BASE="ImpHate")

(21480, 4) (13747, 4) (3437, 4) (4296, 4)
hate_dataset size:  34776
control_dataset size:  47260
hate_set size:  1250
control_set size:  1250
union_set size:  2141
1.010584839089053
1.0105848390890793


 ---- JS
Jenson Shannon Distance for H N 0.12225002324481829


In [11]:
for_other_datasets(BASE="ICDE")

(23748, 4) (15198, 4) (3800, 4) (4750, 4)
hate_dataset size:  8260
control_dataset size:  113607
hate_set size:  1250
control_set size:  1250
union_set size:  2223
1.0078410830848197
1.0078410830848454


 ---- JS
Jenson Shannon Distance for H N 0.2546480162061443


In [13]:
binary_compare("HOPN")
binary_compare("Founta")
binary_compare("davidson")
binary_compare("HASOC19")
binary_compare("OLID")
binary_compare("HateEval")
binary_compare("ImpHate")
binary_compare("ICDE")

(51367, 20) (41093, 20) (5137, 20) (5137, 20)
Here
hate_dataset size:  131175
control_dataset size:  367111
hate_set size:  1250
control_set size:  1250
union_set size:  2137
1.001772500464638
1.0017725004646392


 ---- JS
Jenson Shannon Distance for H N 0.09107749661149464
(59189, 4) (37880, 4) (9471, 4) (11838, 4)
Here
hate_dataset size:  48356
control_dataset size:  126947
hate_set size:  1250
control_set size:  1250
union_set size:  2244
1.0055985175756739
1.0055985175756452


 ---- JS
Jenson Shannon Distance for H N 0.3245336912919928
(24783, 4) (15860, 4) (3966, 4) (4957, 4)
Here
hate_dataset size:  77729
control_dataset size:  13522
hate_set size:  1250
control_set size:  1250
union_set size:  2221
1.010388137624103
1.0103881376241195


 ---- JS
Jenson Shannon Distance for H N 0.3302780655141713
(17657, 4) (11468, 4) (2868, 4) (3321, 4)
hate_dataset size:  31733
control_dataset size:  36899
hate_set size:  1250
control_set size:  1250
union_set size:  2227
1.0137879450740026
1.0

In [21]:
def for_intra_dataset(BASE,prune=True):
    df, train_df, val_df, test_df = get_combined_df(BASE)
    
    column = 'clean_text'
        
    train_dataset = preprocess_dataset(train_df[column].values.tolist(),prune=prune,high_prune=1000,low_prune=1000)
    print("train_dataset size: ", len(train_dataset))
    val_dataset = preprocess_dataset(val_df[column].values.tolist(),prune=prune,high_prune=1000,low_prune=1000)
    print("val_dataset size: ", len(val_dataset))
    test_dataset = preprocess_dataset(test_df[column].values.tolist(),prune=prune,high_prune=1000,low_prune=1000)
    print("test_dataset size: ", len(test_dataset))

    train_set = set(train_dataset)
    print("train_set size: ", len(train_set))
    val_set = set(val_dataset)
    print("val_set size: ", len(val_set))
    test_set = set(test_dataset)
    print("test_set size: ", len(test_set))
    
    union_set = test_set.union(val_set.union(train_set))
    print("union_set size: ", len(union_set))

    train_counter = Counter(train_dataset)
    val_counter = Counter(val_dataset)
    test_counter = Counter(test_dataset)
    
    pdf_train = []
    pdf_test = []
    pdf_val = []
    
    pdf_train_raw = []
    pdf_test_raw = []
    pdf_val_raw = []

    N = len(train_dataset) + len(test_dataset) + len(val_dataset)
    V = len(union_set)

    for word in union_set : 
        if word in train_set : 
            pdf_train.append((train_counter[word]+1)/(len(train_dataset) + len(train_set)))
        else : 
            pdf_train.append(1/(N+V))
        pdf_train_raw.append(train_counter.get(word,0))
        if word in test_set : 
            pdf_test.append((test_counter[word]+1)/(len(test_dataset) + len(test_set)))
        else : 
            pdf_test.append(1/(N+V))
        pdf_test_raw.append(test_counter.get(word,0))
        if word in val_set : 
            pdf_val.append((val_counter[word]+1)/(len(val_dataset) + len(val_set)))
        else : 
            pdf_val.append(1/(N+V))
        pdf_val_raw.append(val_counter.get(word,0))
    print(sum(pdf_train))
    print(sum(pdf_val))
    print(sum(pdf_test))
    
    print("\n\n ---- JS")
    print("Jenson Shannon Distance for Train Val", JSD(pdf_train, pdf_val))
    print("Jenson Shannon Distance for Train Test", JSD(pdf_train, pdf_test))
    print("Jenson Shannon Distance for Val Test", JSD(pdf_val, pdf_test))
    
#     print("\n\n ---- MW")
#     print("2 way MW TEST for Train Val:: ", mannwhitneyu(pdf_train_raw, pdf_val_raw))
#     print("2 way MW TEST  for Train Tes:: ", mannwhitneyu(pdf_train_raw, pdf_test_raw))
#     print("2 way MW TEST  Val Test:: ", mannwhitneyu(pdf_val_raw, pdf_test_raw))

In [22]:
for_intra_dataset("HOPN")

(51367, 20) (41093, 20) (5137, 20) (5137, 20)
train_dataset size:  456613
val_dataset size:  58425
test_dataset size:  58140
train_set size:  2000
val_set size:  2000
test_set size:  2000
union_set size:  4210
1.003827582145793
1.0038275821458011
1.0038275821458271


 ---- JS
Jenson Shannon Distance for Train Val 0.03308766911812824
Jenson Shannon Distance for Train Test 0.032659581917138414
Jenson Shannon Distance for Val Test 0.05181550453670594


In [23]:
for_intra_dataset("Founta")

(59189, 4) (37880, 4) (9471, 4) (11838, 4)
train_dataset size:  140293
val_dataset size:  36056
test_dataset size:  44821
train_set size:  2000
val_set size:  2000
test_set size:  2000
union_set size:  4238
1.0099286626916726
1.0099286626916772
1.0099286626916932


 ---- JS
Jenson Shannon Distance for Train Val 0.053788437832689315
Jenson Shannon Distance for Train Test 0.04846997819102427
Jenson Shannon Distance for Val Test 0.0754203522964162


In [24]:
for_intra_dataset("davidson")

(24783, 4) (15860, 4) (3966, 4) (4957, 4)
train_dataset size:  66452
val_dataset size:  17916
test_dataset size:  21800
train_set size:  2000
val_set size:  2000
test_set size:  2000
union_set size:  4301
1.0208293729462994
1.0208293729463154
1.0208293729462765


 ---- JS
Jenson Shannon Distance for Train Val 0.06771875323871206
Jenson Shannon Distance for Train Test 0.06026124223007431
Jenson Shannon Distance for Val Test 0.08971454087799793


In [25]:
for_intra_dataset("HASOC19")

(17657, 4) (11468, 4) (2868, 4) (3321, 4)
train_dataset size:  54671
val_dataset size:  15225
test_dataset size:  20701
train_set size:  2000
val_set size:  2000
test_set size:  2000
union_set size:  4529
1.0265857914764973
1.0265857914765046
1.0265857914765284


 ---- JS
Jenson Shannon Distance for Train Val 0.1119078369639496
Jenson Shannon Distance for Train Test 0.19135593616269655
Jenson Shannon Distance for Val Test 0.21720498214426892


In [26]:
for_intra_dataset("OLID")

(14100, 4) (10592, 4) (2648, 4) (860, 4)
train_dataset size:  52132
val_dataset size:  14702
test_dataset size:  6646
train_set size:  2000
val_set size:  2000
test_set size:  2000
union_set size:  4470
1.0316869788325564
1.0316869788325191
1.0316869788325425


 ---- JS
Jenson Shannon Distance for Train Val 0.08986306052867511
Jenson Shannon Distance for Train Test 0.20220533295527315
Jenson Shannon Distance for Val Test 0.2182595527729288


In [27]:
for_intra_dataset("HateEval")

(19600, 4) (13500, 4) (1500, 4) (4600, 4)
train_dataset size:  62983
val_dataset size:  9423
test_dataset size:  25855
train_set size:  2000
val_set size:  2000
test_set size:  2000
union_set size:  4568
1.0249734996936448
1.0249734996936748
1.0249734996936624


 ---- JS
Jenson Shannon Distance for Train Val 0.1480268255522918
Jenson Shannon Distance for Train Test 0.18816951767161183
Jenson Shannon Distance for Val Test 0.2588814208053837


In [28]:
for_intra_dataset("ImpHate")

(21480, 4) (13747, 4) (3437, 4) (4296, 4)
train_dataset size:  62303
val_dataset size:  16648
test_dataset size:  20385
train_set size:  2000
val_set size:  2000
test_set size:  2000
union_set size:  4266
1.0218721646299675
1.0218721646299782
1.0218721646299747


 ---- JS
Jenson Shannon Distance for Train Val 0.07558284584057867
Jenson Shannon Distance for Train Test 0.06984425027090946
Jenson Shannon Distance for Val Test 0.10355789091427187


In [29]:
for_intra_dataset("ICDE")

(23748, 4) (15198, 4) (3800, 4) (4750, 4)
train_dataset size:  98607
val_dataset size:  25635
test_dataset size:  31945
train_set size:  2000
val_set size:  2000
test_set size:  2000
union_set size:  4227
1.0138828281821446
1.0138828281821501
1.0138828281821266


 ---- JS
Jenson Shannon Distance for Train Val 0.06335808061195965
Jenson Shannon Distance for Train Test 0.054082222278167356
Jenson Shannon Distance for Val Test 0.08354065514387703


In [22]:
# counter = split_count(hate_dataset)
# # print(' '.join(emoji for emoji in counter))
# mc = Counter(counter)
# print(mc)

In [23]:
# counter = split_count(offensive_dataset)
# # print(' '.join(emoji for emoji in counter))
# mc = Counter(counter)
# print(mc)

In [24]:
# counter = split_count(provocative_dataset)
# # print(' '.join(emoji for emoji in counter))
# mc = Counter(counter)
# print(mc)

In [25]:
# counter = split_count(control_dataset)
# # print(' '.join(emoji for emoji in counter))
# mc = Counter(counter)
# print(mc)

In [26]:
# import pandas as pd

In [1]:
# def binary_compare(BASE1,BASE2, multi=True,prune=True):
#     df1, train_df, val_df, test_df = get_combined_df(BASE1)
#     df2, train_df, val_df, test_df = get_combined_df(BASE2)
#     column = 'clean_text'
#     num_unq = len(df["label"].unique())
#     if num_unq >2 and multi:
#         hate_dataset = preprocess_dataset(df[df['label'] <=1][column].values.tolist(),prune=prune)
#         print("hate_dataset size: ", len(hate_dataset))
#         control_dataset = preprocess_dataset(df[df['label'] > 1][column].values.tolist(),prune=prune)
#         print("control_dataset size: ", len(control_dataset))
#     else:
#         hate_dataset = preprocess_dataset(df[df['label'] == 0][column].values.tolist(),prune=prune)
#         print("hate_dataset size: ", len(hate_dataset))
#         control_dataset = preprocess_dataset(df[df['label'] == 1][column].values.tolist(),prune=prune)
#         print("control_dataset size: ", len(control_dataset))
#     hate_set = set(hate_dataset)
#     print("hate_set size: ", len(hate_set))
#     control_set = set(control_dataset)
#     print("control_set size: ", len(control_set))
#     union_set = control_set.union(hate_set)
#     print("union_set size: ", len(union_set))

#     hate_counter = Counter(hate_dataset)
#     control_counter = Counter(control_dataset)

#     pdf_hate = []
#     pdf_control = []
#     pdf_hate_raw = []
#     pdf_control_raw = []
#     N = len(hate_dataset) + len(control_dataset)
#     V = len(union_set)

#     for word in union_set : 
#         if word in hate_set : 
#             pdf_hate.append((hate_counter[word]+1)/(len(hate_dataset) + len(hate_set)))
#         else : 
#             pdf_hate.append(1/(N+V))
#         pdf_hate_raw.append(hate_counter.get(word,0))
#         if word in control_set : 
#             pdf_control.append((control_counter[word]+1)/(len(control_dataset) + len(control_set)))
#         else : 
#             pdf_control.append(1/(N+V))
#         pdf_control_raw.append(control_counter.get(word,0)) 
#     print(sum(pdf_hate))
#     print(sum(pdf_control))

#     print("\n\n ---- JS")
#     print("Jenson Shannon Distance for H N", JSD(pdf_hate, pdf_control))

In [30]:
 df, train_df, val_df, test_df = get_combined_df("HOPN")

(51367, 20) (41093, 20) (5137, 20) (5137, 20)


In [31]:
df.head()

,id,text,image,uid,label,topic,n_likes,n_retweets,n_u_friends,n_u_followers,n_u_likes,n_upper,neg_score,pos_score,n_hashtags,n_urls,n_emojis,n_mentions,clean_text,created_at
0,817821482160046080,"I haven't eaten at #tacobell in years, but I ...",NaN,17408077,3,Never Trump Campaign,1,1,1028,373,26720,2,0.000,0.000,3,1,0,0,"I haven't eaten at taco bell in years, but I k...",Sat Jan 07 19:53:30 +0000 2017
1,1053729079525023745,@tnewtondunn @johnsweeneyroar really great pos...,NaN,2522923753,1,Delhi Riots 2020,1,0,376,1638,108541,2,0.239,0.000,8,0,0,2,$MENTION$ $MENTION$ really great posters aroun...,Sat Oct 20 19:26:04 +0000 2018
2,456572669802995713,@_UmarKhalid @DMoreland95 @JordiCrookes Hahaaa...,NaN,314662855,1,Delhi Riots 2020,1,0,534,801,2650,0,0.092,0.164,0,0,1,3,"$MENTION$ $MENTION$ $MENTION$ Hahaaa cba, I'm ...",Wed Apr 16 23:19:41 +0000 2014
3,1207127381330268160,#cash #Cashless 1 author encyclopedia site:Mod...,https://pbs.twimg.com/media/C2dtiaXXAAAgQJb.jpg,718130220746846208,3,Demonetization,0,0,0,0,0,0,0.000,0.000,3,2,0,0,cash cashless 1 author encyclopedia site:Modi'...,Wed Dec 18 02:36:09 +0000 2019
4,794159407294717952,God Save the Queen; women and children first! ...,NaN,49362323,1,Brexit,2,2,2564,3940,533,0,0.272,0.000,1,1,1,0,God Save the Queen; women and children first! ...,Thu Nov 03 12:48:52 +0000 2016


In [72]:
emnlp_fig_id_list=[
1233086926242054144,
1232209842825089025,
1158338062805610496,
986964907877744641,
1232733459099136001,
1232591331379286017,
1232770835645005824,
1233235478737911809,
1232615038491316226,
680816856316403712
]

In [81]:
for id_ in emnlp_fig_id_list:
    print("\n\n@@@@")
    print(id_)
    row = df[df["id"]==id_].iloc[0]
    assert row["id"]==id_
    assert row["label"]==0
#     print(row["id"])
    print(row["text"])
#     print(row["uid"])
    print(row["topic"])



@@@@
1233086926242054144
scary! If libbies are still trying to defend peaceful #Tahir_hussain_terrorist you are an islamofacsist too #HinduLivesMatter #SaveHindusFromMuslims #SaveDelhiHindus https://t.co/0yVOG2Z8KX
#HinduLivesMatter


@@@@
1232209842825089025
and these animals are supported by many hindus also....congi and JDS politicians in karnataka...they are blaming police.....selfish pigs have their  own greed to fill...#DelhiRiots #DelhiBurning https://t.co/st6AQi6xg8
Delhi Riots 2020


@@@@
1158338062805610496
Great to see non-experts and laymen praise the situations with #Article370revoked and #Artical35A ! The season of uninformed, biased and majoritarian #WhatsApp messages is upon us, till 2024! This is totally going to work out like #Demonetization did. 

#Hallelujah !
Demonetization


@@@@
986964907877744641
While people reeled under #Demonetisation, BJP's income soared 81%, Amit Shah's son's income soared 16000%. No #BlackMoney recovered! #No2Modinomics which only works 

In [77]:
df[df["id"]==id_].iloc[0]

id                                             1233086926242054144
text             scary! If libbies are still trying to defend p...
image                                                          NaN
uid                                             864404114310135809
label                                                            0
topic                                            #HinduLivesMatter
n_likes                                                          1
n_retweets                                                       0
n_u_friends                                                    133
n_u_followers                                                   17
n_u_likes                                                      112
n_upper                                                          0
neg_score                                                    0.137
pos_score                                                    0.137
n_hashtags                                                    

In [34]:
df["topic"].unique()[0]

'Never Trump Campaign'

In [93]:
def topic_info(topic):
    print("\n@@@@@@@@@@@@@")
    print("topic",topic)
    df_topic=df[df["topic"]==topic]
    assert len(df_topic["topic"].unique())==1
    unq_tids = len(df_topic)
    unq_users = len(set(df_topic["uid"]))
    print("Unq ids and uids", unq_tids,unq_users)
#     print("Label wise ids", df_topic.groupby("label")["id"].count())
    uid0 = set(df_topic[df_topic["label"]==0]["uid"])
    uid1 = set(df_topic[df_topic["label"]==1]["uid"])
    uid2 = set(df_topic[df_topic["label"]==2]["uid"])
    uid3 = set(df_topic[df_topic["label"]==3]["uid"])
    
    print("Label 0 unq ids and uids",len(set(df_topic[df_topic["label"]==0]["id"])),len(uid0))
    print("Label 1 unq ids and uids",len(set(df_topic[df_topic["label"]==1]["id"])),len(uid1))
    print("Label 2 unq ids and uids",len(set(df_topic[df_topic["label"]==2]["id"])),len(uid2))
    print("Label 3 unq ids and uids",len(set(df_topic[df_topic["label"]==3]["id"])),len(uid3))
    _0_ex = uid0.difference(uid1.union(uid2).union(uid3))
    print("EX 0",len(_0_ex))
    _1_ex = uid1.difference(uid0.union(uid2).union(uid3))
    print("EX 1",len(_1_ex))
    _2_ex = uid2.difference(uid1.union(uid0).union(uid3))
    print("EX 2",len(_2_ex))
    _3_ex = uid3.difference(uid1.union(uid2).union(uid0))
    print("EX 3",len(_3_ex))

In [94]:
for topic in df["topic"].unique():
    topic_info(topic)
topic_info("NOTA")


@@@@@@@@@@@@@
topic Never Trump Campaign
Unq ids and uids 2257 1221
Label 0 unq ids and uids 5 5
Label 1 unq ids and uids 273 222
Label 2 unq ids and uids 536 334
Label 3 unq ids and uids 1443 889
EX 0 1
EX 1 130
EX 2 185
EX 3 707

@@@@@@@@@@@@@
topic Delhi Riots 2020
Unq ids and uids 16858 8209
Label 0 unq ids and uids 1503 1142
Label 1 unq ids and uids 2071 1526
Label 2 unq ids and uids 4776 3248
Label 3 unq ids and uids 8508 4795
EX 0 553
EX 1 767
EX 2 1871
EX 3 3188

@@@@@@@@@@@@@
topic Demonetization
Unq ids and uids 6578 3713
Label 0 unq ids and uids 574 495
Label 1 unq ids and uids 915 750
Label 2 unq ids and uids 1446 1154
Label 3 unq ids and uids 3643 2112
EX 0 290
EX 1 485
EX 2 743
EX 3 1601

@@@@@@@@@@@@@
topic Brexit
Unq ids and uids 8127 4466
Label 0 unq ids and uids 38 35
Label 1 unq ids and uids 505 423
Label 2 unq ids and uids 1041 805
Label 3 unq ids and uids 6543 3654
EX 0 17
EX 1 247
EX 2 515
EX 3 3307

@@@@@@@@@@@@@
topic Umar Khalid JNU
Unq ids and uids 5475 3776


In [97]:
unq_tids = len(df["id"])
unq_users = len(set(df["uid"]))
print("Unq ids and uids", unq_tids,unq_users)
uid0 = set(df[df["label"]==0]["uid"])
uid1 = set(df[df["label"]==1]["uid"])
uid2 = set(df[df["label"]==2]["uid"])
uid3 = set(df[df["label"]==3]["uid"])
print("Label 0 unq ids and uids",len(set(df[df["label"]==0]["id"])),len(uid0))
print("Label 1 unq ids and uids",len(set(df[df["label"]==1]["id"])),len(uid1))
print("Label 2 unq ids and uids",len(set(df[df["label"]==2]["id"])),len(uid2))
print("Label 3 unq ids and uids",len(set(df[df["label"]==3]["id"])),len(uid3))
_0_ex = uid0.difference(uid1.union(uid2).union(uid3))
print("EX 0",len(_0_ex))
_1_ex = uid1.difference(uid0.union(uid2).union(uid3))
print("EX 1",len(_1_ex))
_2_ex = uid2.difference(uid1.union(uid0).union(uid3))
print("EX 2",len(_2_ex))
_3_ex = uid3.difference(uid1.union(uid2).union(uid0))
print("EX 3",len(_3_ex))

Unq ids and uids 51367 25796
Label 0 unq ids and uids 3715 2925
Label 1 unq ids and uids 8070 6161
Label 2 unq ids and uids 10556 7088
Label 3 unq ids and uids 29026 16186
EX 0 1455
EX 1 3724
EX 2 3903
EX 3 11928
